In [1]:
!pip install pyspark graphframes

In [2]:
!export PYSPARK_SUBMIT_ARGS='--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12'

In [3]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import tensorflow_hub as hub
from pyspark.sql import functions as f
from pyspark.sql import types as t
from pyspark.sql import Window as w
from graphframes import GraphFrame
from pyspark.ml.linalg import DenseVector, SparseVector
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, RegexTokenizer, CountVectorizer, StopWordsRemover, NGram, Normalizer, VectorAssembler, Word2Vec, Word2VecModel, PCA
from pyspark.ml.clustering import LDA
from pyspark.ml.linalg import VectorUDT, Vectors

In [4]:
spark = (
    SparkSession.builder
    .config('spark.executor.memory', '4g')
    .config('spark.app.name', 'Spark Updated Conf')
    .config('spark.executor.cores', '2')
    .config('spark.cores.max', '2')
    .config('spark.driver.memory','8g')
    .getOrCreate()
)

# 0. Load data

In [5]:
df = spark.read.csv("./data/X2.csv", header=True)
df.toPandas()

,instance_id,brand,cpu_brand,cpu_model,cpu_type,cpu_frequency,ram_capacity,ram_type,ram_frequency,hdd_capacity,ssd_capacity,weight,dimensions,title
0,www.softwarecity.ca//737,Lenovo,Intel. i5-3320M,i5-3320M,Dual-core ( 2 Core ). Core i5,2.60 GHz,None,DDR3 SDRAM. DDR3-1600/PC3-12800. DDR3 SDRAM,DDR3-1600/PC3-12800,320 GB,None,1.80 kg,None,"""Lenovo Thinkpad X230 34352jf Tablet Pc - 12.5..."
1,www.isupplyhub.com//1256,Acer,1.6 GHz Intel Core i5-4200U. Intel Core I5,None,1.6 GHz Intel Core i5-4200U,1.6 GHz Intel Core i5-4200U,8 GB DDR3L SDRAM,DDR3 SDRAM. 8 GB DDR3L SDRAM,None,500 GB mechanical_hard_drive,None,4.8 pounds,15.02 x 10.08 x 0.90 inches,Amazon.com : Acer Aspire V7-582PG-6479 15.6-In...
2,www.isupplyhub.com//326,Acer,1.6 GHz Intel Core i5. Intel Core I5,None,1.6 GHz Intel Core i5,1.6 GHz Intel Core i5,4 GB DDR3-SDRAM,DDR3 SDRAM. 4 GB DDR3-SDRAM,None,500 GB mechanical_hard_drive,None,5.2 pounds,15.02 x 10.08 x 1 inches,Amazon.com : Acer Aspire E1-572-6870 15.6 Inch...
3,www.isupplyhub.com//821,HP,None,None,None,None,4 GB SDRAM DDR3,DDR3 SDRAM. 4 GB SDRAM DDR3,None,500 GB,None,4.8 pounds,15.18 x 0.89 x 10.16 inches,"""Amazon.com : 15.6"""" HP 15-f009wm Amd Dual-Cor..."
4,www.isupplyhub.com//157,Asus,1.7 GHz Core i5-3317U. Intel,None,1.7 GHz Core i5-3317U,1.7 GHz Core i5-3317U,4 GB DDR3,DDR3 SDRAM. 4 GB DDR3,None,256 MB,None,2.9 pounds,8.80 x 0.70 x 12.80 inches,Amazon.com : ASUS UX31A-XB52 13.3-Inch Ultrabo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,www.vology.com//873,Lenovo ThinkPad X230 2320 - 12.5 '' - Core i5 ...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,4 GB DDR3 Slots Qty 2 Empty Slots 1 Max RAM Su...,4 GB DDR3 Slots Qty 2 Empty Slots 1 Max RAM Su...,4 GB DDR3 Slots Qty 2 Empty Slots 1 Max RAM Su...,180 GB SSD. 180 GB SSD. Lenovo ThinkPad X230 2...,180 GB SSD. 180 GB SSD,3.3 lbs 3.3 lbs,8.1 in. 12 in x 8.1 in x 1 in. 1 in. 12 in,"""Lenovo ThinkPad X230 2320 - 12.5"""" - Core i5 ..."
339,www.vology.com//823,Lenovo ThinkPad X230 2325 - 12.5 '' - Core i5 ...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,4 GB DDR3 Slots Qty 2 Max RAM Supported 16 GB ...,4 GB DDR3 Slots Qty 2 Max RAM Supported 16 GB ...,4 GB DDR3 Slots Qty 2 Max RAM Supported 16 GB ...,500 GB HDD / 7200 rpm. 500 GB HDD / 7200 rpm. ...,500 GB HDD / 7200 rpm. 500 GB HDD / 7200 rpm,3.3 lbs 3.3 lbs,8.1 in. 12 in x 8.1 in x 1 in. 1 in. 12 in,"""Lenovo ThinkPad X230 2325 - 12.5"""" - Core i5 ..."
340,www.vology.com//2723,Lenovo ThinkPad X230 Tablet 3438 - 12.5 '' - C...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,500 GB HDD / 7200 rpm. 500 GB HDD / 7200 rpm. ...,500 GB HDD / 7200 rpm. 500 GB HDD / 7200 rpm,4 lbs 4 lbs,9 in. 12 in x 9 in x 1.2 in. 1.2 in. 12 in,"""Lenovo ThinkPad X230 Tablet 3438 - 12.5"""" - C..."
341,www.vology.com//1349,Lenovo ThinkPad X230 2324 - 12.5 '' - Core i5 ...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,320 GB HDD / 7200 rpm. 320 GB HDD / 7200 rpm. ...,320 GB HDD / 7200 rpm. 320 GB HDD / 7200 rpm,3.3 lbs 3.3 lbs,8.1 in. 12 in x 8.1 in x 1 in. 1 in. 12 in,"""Lenovo ThinkPad X230 2324 - 12.5"""" - Core i

# 0. Data cleaning

In [6]:
"""
Job:
    1 - remove ssd capacity, dimensions
    2 - infer brand from title
    3 - infer cpu brand and type
    4 - uniform weights
"""
# Set everything to lowercase
for c in df.columns:
    df = df.withColumn(c, f.lower(f.col(c)))

df = df.drop('ssd_capacity')
df = df.drop('dimensions')
# Extract brand or infer from title
df = df.withColumn('brand', f.regexp_extract('brand', "^(\w+)", 0))
computer_brands = ['lenovo', 'acer', 'hp', 'dell', 'asus', 'samsung', 'huawei', 'surface', 'apple']
computer_brands_pattern = '({})'.format('|'.join(computer_brands))
df = df.withColumn('brand', f.when( f.regexp_extract('title', computer_brands_pattern, 0)!='', f.regexp_extract('title', computer_brands_pattern, 0))\
                   .otherwise(df.brand))
#exctract cpu_brand and infer type if intel
cpu_brands = ['intel', 'apple', 'amd', 'nvidia', 'arm']
cpu_pattern = '({})'.format('|'.join(cpu_brands))
df = df.withColumn('cpu_model',f.regexp_extract('cpu_model', '(i\d|pentium|celeron|a\d)', 0))
df = df.withColumn('cpu_model', f.when( (f.regexp_extract('cpu_brand','(intel|amd)', 0 )!='') & f.isnull(df.cpu_model) ,\
                                        f.regexp_extract('cpu_brand', '(i\d|pentium|celeron|a\d)', 0))\
                   .otherwise(df.cpu_model))
df = df.withColumn('cpu_brand', f.when(f.regexp_extract('cpu_brand', cpu_pattern, 0) != '', f.regexp_extract('cpu_brand', cpu_pattern, 1))\
                                       .otherwise(f.regexp_extract('title', cpu_pattern, 0)))
#convert weight from pounds to kilos
df = df.withColumn('weight', f.when(df.weight.contains('pounds') | df.weight.contains('lbs'),
                                    (f.regexp_extract('weight', '(\d+.?\d)', 0).cast(t.DoubleType()))).otherwise(
                                    f.round(f.regexp_extract('weight', '(\d+.?\d)', 0).cast(t.DoubleType())*2.20462,1)
                        )
                    )

In [7]:
"""
merge ram columns and cpu columns into one for ram and one for cpu
"""
#from more_itertools import intersperse

def merge_columns(df, column_names, output):
    df = df.withColumn(output, f.concat_ws(" ", *column_names))
    return df.drop(*column_names)

ddf = df.drop("ram_frequency")
ddf = merge_columns(ddf, ["cpu_brand", "cpu_model", "cpu_frequency", "cpu_type"], "cpu")
ddf = merge_columns(ddf, ["ram_capacity", "ram_type"], "ram")
ddf.columns

['instance_id', 'brand', 'hdd_capacity', 'weight', 'title', 'cpu', 'ram']

# 1. Blocking
Blocking will be done feeding a TF-IDF matrix to an LDA model and extracting
keywords from the title matching them to topics.

In [8]:
@f.udf(returnType=t.ArrayType(t.StringType()))
def filter_alnum(arr):
    return [t for t in arr if t.isalnum() and len(t) > 2]

"""Returns the df with tokenized columns with stopwords removed"""
def tokenize(df, string_cols):
    output = df
    stopW = ['softwarecity', 'amazon', 'com', 'pc', 'windows', 'computers', 'computer', 'accessories', 'laptop', 'notebook', 'kg', 'inch', 'processor', 'memory','gb', 'ram', 'hdd', 'ssd', 'cpu', 'display', 'hz', 'ghz', 'tb','rpm', 'slot', 'slots', 'mhz', 'cache', 'ram', 'ddram', 'dram', 'hd']
    for c in string_cols:
        output = output.withColumn('temp', f.coalesce(f.col(c), f.lower(c), f.lit('')))
        tokenizer = RegexTokenizer(inputCol='temp', outputCol=c+"_rawtokens", pattern = "\\W")
        remover = StopWordsRemover(inputCol=c+"_rawtokens", outputCol=c+"_tokens", stopWords=stopW)

        output = tokenizer.transform(output)
        output = remover.transform(output).drop(c+"_rawtokens")
        output = output.withColumn(c+'_tokens', f.array_distinct(filter_alnum(f.col(c+"_tokens"))))
    # output has c+tokens columns
    return output.drop("temp")

def generate_blocking_keys(df, token_cols, min_freq=1):
    """Pipeline:
            1 - CountVectorizer -> TF
            2 - IDF
            3 - LDA
    """
    # merge all tokens in one column + merge title and brand for blocking
    df = df.withColumn('tokens', f.array_distinct(f.concat(*token_cols)))\
        .drop('cpu_tokens')\
        .drop('ram_tokens')\
        .drop('hdd_capacity_tokens')\
        .drop('weight_tokens')\
        .withColumn('tokens_for_vocab', f.array_distinct(f.concat('title_tokens', 'brand_tokens')))\
        .drop('title_tokens')\
        .drop('brand_tokens')
    cv = CountVectorizer(inputCol='tokens_for_vocab', outputCol="raw_features")
    cvmodel = cv.fit(df)
    df_vect = cvmodel.transform(df)
    df_vect = df_vect.drop('tokens_for_vocab')

    idf = IDF(inputCol="raw_features", outputCol="features", minDocFreq=min_freq)
    idfModel = idf.fit(df_vect)
    df_idf= idfModel.transform(df_vect)

    normalizer = Normalizer(p=2.0, inputCol='features', outputCol='tfidf')
    output = normalizer.transform(df_idf)
    output = output.drop('raw_features').drop('features')
    k = output.select('brand').distinct().count()
    lda = LDA(k=k, maxIter=1000, featuresCol='tfidf')
    lda_model = lda.fit(output)
    vocab = cvmodel.vocabulary

    #returns words for each topic term
    @f.udf(returnType=t.ArrayType(t.StringType()))
    def get_words(token_list):
        return [vocab[token_id] for token_id in token_list]

    #create list of topic keywords
    # i.e topic 1 -> acer, anspire, intel
    topics = lda_model.describeTopics(3).withColumn('topicWords', get_words(f.col('termIndices'))).collect()
    list_of_topics = []
    for r in topics:
        topicW = r.__getitem__('topicWords')
        for w in topicW:
            list_of_topics.append(w)

    #returns list of 3 'hashtags' i.e keywords for topic
    #from tokens: title, brand, cpu_brand
    @f.udf(returnType=t.ArrayType(t.StringType()))
    def get_key(words):
        l = [w for w in words if w in list_of_topics]
        l = list(set(l))
        l.sort()
        return l[:3]
    output = output.withColumn("blocking_key", get_key(f.col("tokens")))
    return output

"""Use universal sentence encoder from tensorflow_hub"""
MODEL = None
def get_model_magic():
  global MODEL
  if MODEL is None:
      MODEL = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
  return MODEL
@f.udf(returnType=VectorUDT())
def encode_sentence(x):
  model = get_model_magic()
  emb = model([x]).numpy()[0]
  return Vectors.dense(emb)

In [9]:
columns = ['title', 'brand', 'cpu', 'ram', 'hdd_capacity', 'weight']
#Generate Blocking Keys

blocking_df = tokenize(ddf, columns)
blocking_df = generate_blocking_keys(blocking_df, [c+'_tokens' for c in columns])
blocking_df.limit(1).show()

#columns: [id, brand, hdd_cap , weight, title, cpu, ram, tokens, tfidf, block_key]

+--------------------+------+------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+
|         instance_id| brand|hdd_capacity|weight|               title|                 cpu|                 ram|              tokens|               tfidf| blocking_key|
+--------------------+------+------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+
|www.softwarecity....|lenovo|      320 gb|   4.0|"lenovo thinkpad ...|intel i5 2.60 ghz...|ddr3 sdram. ddr3-...|[lenovo, thinkpad...|(314,[0,1,2,4,6,7...|[3320m, x230]|
+--------------------+------+------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+



In [10]:
#Add encoding on all the columns (not tokenized)
cols_to_encode = columns
blocking_df = blocking_df.withColumn('to_encode', f.concat(*cols_to_encode))
blocking_df = blocking_df.withColumn('encoding', encode_sentence(f.coalesce(f.col('to_encode'), f.lit(''))))\
    .drop('to_encode')

blocking_df.limit(1).show()

+--------------------+------+------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+
|         instance_id| brand|hdd_capacity|weight|               title|                 cpu|                 ram|              tokens|               tfidf| blocking_key|            encoding|
+--------------------+------+------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+
|www.softwarecity....|lenovo|      320 gb|   4.0|"lenovo thinkpad ...|intel i5 2.60 ghz...|ddr3 sdram. ddr3-...|[lenovo, thinkpad...|(314,[0,1,2,4,6,7...|[3320m, x230]|[0.03184049949049...|
+--------------------+------+------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+



In [11]:
blocking_df.groupby('blocking_key').count().show()

+--------------------+-----+
|        blocking_key|count|
+--------------------+-----+
|       [touchscreen]|   10|
|         [elitebook]|   20|
|                  []|   54|
|[cool, silver, to...|    2|
|              [x220]|    4|
|             [3320m]|    1|
|[3320m, 8570p, el...|    3|
|            [silver]|    9|
|  [2760p, elitebook]|    2|
|            [carbon]|   73|
|       [3320m, x230]|  110|
|        [4291, x220]|    4|
|      [cool, silver]|    3|
|  [2170p, elitebook]|    3|
|[silver, touchscr...|    1|
|              [x230]|   27|
| [cool, touchscreen]|    3|
|               [771]|   11|
|   [160, upgradable]|    2|
| [elitebook, silver]|    1|
+--------------------+-----+



# 2. Candidate pairs generation and match likelihood

In [12]:
"""
This cell output a candidates dataframe that has
instance_ids pairs that makes sense to compare, i.e each
entity will be paired with another entity from the same block
"""
# Filter blocks to only keep ones bigger than one
pairs = (
    blocking_df
    .groupby('blocking_key').agg(f.count('instance_id').alias('size'), f.collect_set('instance_id').alias('id'))\
    .filter(f.col('size') > 1).select('blocking_key',f.explode('id').alias('id'))
)
left = pairs.withColumnRenamed('id', 'src')
right = pairs.withColumnRenamed('id', 'dst')
#candidates based on matching of blocking_key (i.e inside the block)
candidates = left.join(right, ['blocking_key'], 'inner')\
    .filter(f.col('src') < f.col('dst'))\
    .select('src', 'dst').distinct()
node = blocking_df.withColumnRenamed('instance_id', 'id')
node.limit(1).show()

+--------------------+------+------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+
|                  id| brand|hdd_capacity|weight|               title|                 cpu|                 ram|              tokens|               tfidf| blocking_key|            encoding|
+--------------------+------+------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+
|www.softwarecity....|lenovo|      320 gb|   4.0|"lenovo thinkpad ...|intel i5 2.60 ghz...|ddr3 sdram. ddr3-...|[lenovo, thinkpad...|(314,[0,1,2,4,6,7...|[3320m, x230]|[0.03184049949049...|
+--------------------+------+------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+



In [13]:
"""
Read label.csv and expand it trough transitivity
"""
labels_col = ['lid', 'rid', 'label']
labels = (
    spark.read.csv("./data/Y2.csv", header=True)
    .withColumnRenamed('left_instance_id', 'lid')
    .withColumnRenamed('right_instance_id', 'rid')
)
"""
n_ones_old = labels.filter('label==1').count()
n_zeros = labels.filter('label==0').count()

tuples = labels.filter('label==1').drop('label').collect()
print(tuples)
ll = []
for t in tuples:
    left = t[0]
    right = t[1]
    if not ll:
        #First initialization
        l = []
        l.append(left)
        l.append(right)
        ll.append(l)
    #loops all the list to find where to put
    found_l = -1
    found_r = -1
    put = False

    for i,l in enumerate(ll):
        #store list indexes where occurance happened
        if left in l:
            found_l = i
        if right in l:
            found_r = i

    if found_l != -1:
        if found_r == -1:
            ll[found_l].append(right)
            put = True
    if found_r != -1:
        if found_l == -1:
            ll[found_r].append(left)
            put = True

    if found_r != -1 and found_l != -1 and found_l != found_r:
        #create new list of list and merge entries
        ll_new = []
        if found_l <= found_r:
            minf = found_l
            maxf = found_r
        else:
            minf = found_r
            maxf = found_l

        for k in range(0, len(ll)):
            if k != maxf:
                ll_new.append(ll[k])
            if k == minf:
                for tp in ll[maxf]:
                    if tp not in ll_new[minf]:
                        ll_new[minf].append(tp)
        ll = ll_new
        put = True
    if found_r == -1 and found_l == -1:
        #new list
        l = []
        l.append(left)
        l.append(right)
        ll.append(l)
#SAFETY CHECK OK
for i,l in enumerate(ll):
    for t in l:
        for j in range(0,len(ll)):
            if j != i:
                if t in ll[j]:
                    print('NOOO')
"""

"\nn_ones_old = labels.filter('label==1').count()\nn_zeros = labels.filter('label==0').count()\n\ntuples = labels.filter('label==1').drop('label').collect()\nprint(tuples)\nll = []\nfor t in tuples:\n    left = t[0]\n    right = t[1]\n    if not ll:\n        #First initialization\n        l = []\n        l.append(left)\n        l.append(right)\n        ll.append(l)\n    #loops all the list to find where to put\n    found_l = -1\n    found_r = -1\n    put = False\n\n    for i,l in enumerate(ll):\n        #store list indexes where occurance happened\n        if left in l:\n            found_l = i\n        if right in l:\n            found_r = i\n\n    if found_l != -1:\n        if found_r == -1:\n            ll[found_l].append(right)\n            put = True\n    if found_r != -1:\n        if found_l == -1:\n            ll[found_r].append(left)\n            put = True\n\n    if found_r != -1 and found_l != -1 and found_l != found_r:\n        #create new list of list and merge entries\n   

In [14]:
"""
from itertools import combinations

labels = labels.filter('label==0')
for l in ll:
    for c in combinations(l,2):
        left = c[0]
        right = c[1]
        nR = spark.createDataFrame([(left, right, '1')], labels_col)
        labels = labels.union(nR)

n_ones_new = labels.filter('label==1').count()
print(f'Ones before expansion: {n_ones_old}')
print(f'Ones after expansion: {n_ones_new}')
print(f'One to zero ratio: {n_ones_new/ n_zeros}')o \
"""

"\nfrom itertools import combinations\n\nlabels = labels.filter('label==0')\nfor l in ll:\n    for c in combinations(l,2):\n        left = c[0]\n        right = c[1]\n        nR = spark.createDataFrame([(left, right, '1')], labels_col)\n        labels = labels.union(nR)\n\nn_ones_new = labels.filter('label==1').count()\nprint(f'Ones before expansion: {n_ones_old}')\nprint(f'Ones after expansion: {n_ones_new}')\nprint(f'One to zero ratio: {n_ones_new/ n_zeros}')o "

In [27]:
"""
Reuse this cell to join a <left_id,right_id> with node to extract features
"""

columns = ["id", "brand", "hdd_capacity" , "weight", "title", "cpu", "ram", "tokens", "tfidf", "blocking_key", "encoding"]
matching_pairs = candidates.join(node.alias("node_1"), candidates.src == node.id, 'inner').drop('id')
for c in columns[1:]:
    matching_pairs = matching_pairs.withColumnRenamed(c, 'l_'+c)

matching_pairs = matching_pairs.alias('one').join(node.alias("node_2"), matching_pairs.dst == node.id, 'inner').drop('id')
for c in columns[1:]:
    matching_pairs = matching_pairs.withColumnRenamed(c, 'r_'+c)
matching_pairs = matching_pairs.withColumnRenamed('src', 'lid').withColumnRenamed('dst','rid')
print(matching_pairs.columns)
#label_df = labels.join(candidates.withColumnRenamed('src','lid').withColumnRenamed('dst','rid'), ['lid','rid'], 'inner')
label_df = labels.join(node.alias("node_1"), labels.lid == node.id, 'inner').drop('id')
for c in columns[1:]:
    label_df = label_df.withColumnRenamed(c, 'l_'+c)

label_df = label_df.alias('one').join(node.alias("node_2"), label_df.rid == node.id, 'inner').drop('id')
for c in columns[1:]:
    label_df = label_df.withColumnRenamed(c, 'r_'+c)
print(label_df.columns)

['lid', 'rid', 'l_brand', 'l_hdd_capacity', 'l_weight', 'l_title', 'l_cpu', 'l_ram', 'l_tokens', 'l_tfidf', 'l_blocking_key', 'l_encoding', 'r_brand', 'r_hdd_capacity', 'r_weight', 'r_title', 'r_cpu', 'r_ram', 'r_tokens', 'r_tfidf', 'r_blocking_key', 'r_encoding']
['lid', 'rid', 'label', 'l_brand', 'l_hdd_capacity', 'l_weight', 'l_title', 'l_cpu', 'l_ram', 'l_tokens', 'l_tfidf', 'l_blocking_key', 'l_encoding', 'r_brand', 'r_hdd_capacity', 'r_weight', 'r_title', 'r_cpu', 'r_ram', 'r_tokens', 'r_tfidf', 'r_blocking_key', 'r_encoding']


In [28]:
import numpy as np
"""
Add some distances as new possible features
"""

@f.udf(returnType=t.DoubleType())
def dot(x, y):
  if x is not None and y is not None:
    return float(x.dot(y))
  else:
    return 0

"""
Compute the similarity as inner product between
left and right encodings
"""
@f.udf(returnType=t.DoubleType())
def encoding_sim(le, re):
    return np.inner(le.toArray(), re.toArray()).item()

def levenshtein_sim(c1, c2):
  output = f.when(f.col(c1).isNull() | f.col(c2).isNull(), 0)\
            .otherwise(1 - f.levenshtein(c1, c2) / f.greatest(f.length(c1), f.length(c2)))
  return output

def num_sim(c1, c2):
  output = f.when(f.col(c1).isNull() | f.col(c2).isNull(), 0)\
            .when((f.col(c1) == 0) & (f.col(c2) == 0), 1)\
            .when((f.col(c1) == 0) | (f.col(c2) == 0), 0)\
            .otherwise(1 - f.abs(f.col(c1) - f.col(c2)) / f.greatest(c1, c2))
  return output

def token_overlap(c1, c2):
  # is the overlap a significant part of the shorter string
  output = f.when(f.col(c1).isNull() | f.col(c2).isNull(), 0)\
            .when((f.size(f.array_distinct(c1)) == 0) | (f.size(f.array_distinct(c2)) == 0), 0)\
            .otherwise(f.size(f.array_intersect(c1, c2)) / f.least(f.size(f.array_distinct(c1)), f.size(f.array_distinct(c1))))
  return output

def calc_sim(df):
    metrics = []
    col_lev = ["brand", "hdd_capacity", "title", "cpu", "ram"]

    for c in columns[1:]:
        if c in col_lev:
            df = df.withColumn(c+'_lev', levenshtein_sim('l_'+c, 'r_'+c))
            metrics.append(c+'_lev')
        if c == 'tokens':
            df = df.withColumn('tokens_overlap', token_overlap('l_tokens', 'r_tokens'))
            metrics.append('tokens_overlap')
        if c == 'weight':
            df = df.withColumn('weight_sim', num_sim('l_weight','r_weight'))
            metrics.append('weight_sim')
        if c == 'tfidf':
            df = df.withColumn('tfidf_sim', dot('l_tfidf','r_tfidf'))
            metrics.append('tfidf_sim')
        if c == 'encoding':
            df = df.withColumn('encoding_sim', encoding_sim('l_encoding','r_encoding'))
            metrics.append('encoding_sim')
    @f.udf(returnType=t.DoubleType())
    def sum_distance(distances):
        return sum(d for d in distances if d != None)

    df = df.withColumn('sum_sim', sum_distance(f.array(*metrics)))
    udf_norm = f.udf(lambda d : d / len(metrics))
    df = df.withColumn('overall_sim', udf_norm(f.col('sum_sim'))).drop(f.col('sum_sim'))

    return df

matching_pairs = calc_sim(matching_pairs)
label_df = calc_sim(label_df)
label_df.limit(1).show()
matching_pairs.limit(1).show()

+--------------------+--------------------+-----+-------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+--------------+--------+--------------------+--------------------+---------+--------------------+--------------------+--------------+--------------------+---------+------------------+----------+------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+
|                 lid|                 rid|label|l_brand|      l_hdd_capacity|l_weight|             l_title|               l_cpu|               l_ram|            l_tokens|             l_tfidf|l_blocking_key|          l_encoding|r_brand|r_hdd_capacity|r_weight|             r_title|               r_cpu|    r_ram|            r_tokens|             r_tfidf|r_blocking_key|          r_encoding|brand_lev|  hdd_capacity_lev|weight_sim|  

In [ ]:
@f.udf(returnType=VectorUDT())
def toList(row):
    l = []
    for v in row:
        for n in v:
            l.append(float(n))
    return Vectors.dense(l)

to_double = f.udf(lambda x: x.toArray()[0].item(), "float") #item converts numpy.float to py float

label_df = label_df.withColumn('features', toList(f.array('l_encoding', 'r_encoding', 'sim')))
label_df = label_df.withColumn('simil', to_double(f.col('sim')))
p = label_df.filter("label==1 AND simil>0.5").count() / label_df.filter('simil>0.5').count()
r = label_df.filter("label==1 AND simil>0.5").count() / label_df.filter(f.col('label')==1).count()
f1 = 2*p*r/(p+r)
print("F1 score: ", f1)
label_df = label_df.withColumn('label', f.col('label').cast(t.IntegerType()))

matching_pairs = matching_pairs.\
    withColumn('simil', to_double('sim')).\
    filter('simil > 0.5').\
    drop('simil').\
    withColumn('features', toList(f.array('l_encoding', 'r_encoding', 'sim')))\
    .drop('l_encoding', 'r_encoding', 'sim')

matching_pairs = matching_pairs.withColumnRenamed('src', 'lid').withColumnRenamed('dst','rid')

# 3. Machine Learning Magic Bitch

In [ ]:
from pyspark.ml.classification import LinearSVC, LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
#model = LinearSVC(featuresCol='features', labelCol='label', weightCol='weights',maxIter=100)
model = LogisticRegression(featuresCol='features', labelCol='label')
param_grid = ParamGridBuilder().addGrid(model.regParam, [0.5, 0.4, 0.3, 0.2, 0.1]).build()
cvs = CrossValidator(estimator=model,
                           estimatorParamMaps=param_grid,
                           evaluator=BinaryClassificationEvaluator(),#(rawPredictionCol='prediction', labelCol='label'),\
                           numFolds=8)

In [ ]:
#Set weights
w = 0.04
label_df = label_df.withColumn('weights', f.when(f.col('label')==0, w)\
                               .otherwise(1.0))
training_set, test_set = label_df.randomSplit([0.8, 0.2])

In [ ]:
#grid_search, hyperpar tuning...
estimator = cvs.fit(training_set)

In [ ]:
prediction = estimator.transform(test_set).select('lid','rid','label','prediction')

In [ ]:
prediction.groupby("prediction").count().toPandas()

estimator.save("model.model")
!zip -r model.zip model.model

accuracy = prediction.filter(f.col('label')==f.col('prediction').cast(t.IntegerType())).count() / prediction.count()
print("Accuracy: ", accuracy)
p = prediction.filter("label==1 AND prediction==1").count() / prediction.filter('prediction==1').count()
r = prediction.filter("label==1 AND prediction==1").count() / prediction.filter('label == 1').count()
f1 = 2*p*r/(p+r)
print("F1 score: ", f1)